In [3]:
from os.path import exists

try:
    from pyomo.environ import *
except:
    %pip install pyomo
from pyomo.environ import *
from pyomo.dae import *


In [6]:
import os
os.getcwd()

'/Users/cchin/Dropbox (MIT)/Routing_Prioritization_Auctions'

In [2]:
from helpers import *
from grid import *
from agent import *
from scenarios import *
from simulate import *

ModuleNotFoundError: No module named 'helpers'

In [3]:
# grid1, agents1, schedule1 = create_hubspoke(num_agents = [25, 25, 25, 25, 25, 25], radius = 7, seed = 10)

# grid2, agents2, schedule2 = create_hubspoke(num_agents = [25, 25, 25, 25, 25, 25], radius = 7, seed = 10)

# print(agents1 == agents2)
# print(schedule1)
# print(schedule2)

False
{1: [<agent.Agent object at 0x29646d070>, <agent.Agent object at 0x29646dcd0>, <agent.Agent object at 0x29646d970>, <agent.Agent object at 0x1082250d0>, <agent.Agent object at 0x29645df10>, <agent.Agent object at 0x296456550>], 2: [<agent.Agent object at 0x29646d5b0>, <agent.Agent object at 0x29646d820>, <agent.Agent object at 0x1082251f0>, <agent.Agent object at 0x29645df70>, <agent.Agent object at 0x2964565b0>, <agent.Agent object at 0x296456f70>], 3: [<agent.Agent object at 0x29646d550>, <agent.Agent object at 0x29646d7c0>, <agent.Agent object at 0x29645d6a0>, <agent.Agent object at 0x29645dfd0>, <agent.Agent object at 0x296456610>, <agent.Agent object at 0x296456fd0>], 4: [<agent.Agent object at 0x10a01ff70>, <agent.Agent object at 0x29646d760>, <agent.Agent object at 0x29645d760>, <agent.Agent object at 0x29645d580>, <agent.Agent object at 0x296456670>, <agent.Agent object at 0x296456220>], 5: [<agent.Agent object at 0x10a3a7640>, <agent.Agent object at 0x29646d4c0>, <agent.

In [3]:
# grid, agents, schedule = create_hubspoke(num_agents = [25, 25, 25, 25, 25, 25], radius = 7, seed = 10)
# grid, agents, schedule = create_connected(radius = 3)
# grid, agents, schedule = create_single()
grid, agents, schedule = create_random(num_agents = 5, radius = 2, iters=10, seed = 2)
# grid, agents, schedule = create_random(num_agents=124, radius=7, iters=50, operator_flag=True, seed = 100)
# grid, agents, schedule = create_random(num_agents=50, radius=7, iters=50, operator_flag=True, seed = 100)

# agents = []
# for i in range(2):
#     agents.append(Agent(Hex(0, 0, 0), Hex(0, -1, 1), 1, 0))
# grid, agents, schedule = create_custom(agents, radius = 2)

for i, ag in enumerate(agents):
    ag._id = i
    # ag._var_cost = 1
    # print(ag._steps[0], ag.loc)
#     ag.move((ag._steps[0], 0))
rev, delay, std_delay, _, _, _, _, _, _, _, _, _ = simulate(grid, agents, schedule, vis= False, prior="backpressure", output=True, debug=False)


Time:  12
Priority:  backpressure
Agent  0 operator  None  costed  (0, 0) finished True departure, scheduled, arrived 0 3 3 price 3
Agent  1 operator  None  costed  (0, 0) finished True departure, scheduled, arrived 3 5 5 price 1
Agent  2 operator  None  costed  (7, 0) finished True departure, scheduled, arrived 6 11 11 price 7
Agent  3 operator  None  costed  (0, 1) finished True departure, scheduled, arrived 7 11 12 price 1
Agent  4 operator  None  costed  (0, 0) finished True departure, scheduled, arrived 1 5 5 price 6
Total Revenue:  7
Total Conflicts:  1
Total Delay:  1
Std Dev Delay:  0.4
Total Pay Costs:  7
Total Wait Costs:  1
Std Dev Normalized Delay:  0.4
Total Normalized Pay Costs:  1.0
Total Normalized Wait Costs:  1.0


In [4]:
print(delay)

1


In [5]:
# print(schedule)
for i in range(len(agents)):
    print(len(agents[i].steps), agents[i].scheduled)

3 3
2 5
5 11
4 11
4 5


In [13]:


# # agents = []
# # for i in range(2):
# #     agents.append(Agent(Hex(0, 0, 0), Hex(0, -1, 1), 1, 0))
# # grid, agents, schedule = create_custom(agents, radius = 2)

# for i, ag in enumerate(agents):
#     ag._id = i

def optimal_solver(grid, agents, schedule, Tf = 20, weights=False):
    Tf = Tf

    # Create variables w for all flights, times, sectors 
    # variables have Agent, Sector (only on path), Time (only after takeoff), Layer (0, 1, 2)
    # for each flight iterate
    vars = []
    departures = {}     # hex : id  where hex is the takeoff and id is agent
    travel = {}         # hex: [(id, next, time, layer)] where next is next hex and time is the first time it's active and layer is the next step layer we care about

    # Variable definition
    # define only for sectors agent will travel in, including start and end
    # define for all times after agent departure
    for ag in agents:
        indices = []

        for i, h in enumerate(ag.steps):

            # create entries in travel
            if h not in travel:
                travel[h] = []
            if i + 1 >= len(ag.steps):
                travel[h].append((ag.id, ag.steps[-1], ag.depart, 2))     ## CHANGE ag.depart to have logic when # var defs are shrunk
            else:
                travel[h].append((ag.id, ag.steps[i+1], ag.depart, 1))


            # create the variables  
            # ## NOTE when # var defs shrunk, must define variable one time-step before it actually can be filled, so that capacity constraints make sense
            for t in range(ag.depart, Tf+1):
                indices.append((ag.id, h, t, 1))

        for t in range(ag.depart, Tf+1): 
            # start and end layers
            indices.append((ag.id, ag.steps[0], t, 0))
            indices.append((ag.id, ag.steps[-1], t, 2))

        vars += indices   

    ### Create model
    m = ConcreteModel()
    m.vars = Set(initialize = vars)
    m.w = Var(m.vars, domain=Binary)    # indexing is (id, q, r, s, time, layer)


    ## set boundary conditions - initialization, ending, and everything layer 1 and 2 at time 0 is 0
    m.boundary = ConstraintList()
    for ag in agents:
        # initialization - enters layer 0
        m.boundary.add(m.w[ag.id, ag.steps[0], ag.depart, 0] == 1)

        # ending - has to be in layer 2 by the end
        m.boundary.add(m.w[ag.id, ag.steps[-1], Tf, 2] == 1)

        # initial empty - all the 1's are not filled when the agent departs
        # the first 1 that exist basically
        for s in ag.steps:
            m.boundary.add(m.w[ag.id, s, ag.depart, 1] == 0)
        
        # initial empty - 2's are not filled when the agent departs
        m.boundary.add(m.w[ag.id, ag.steps[-1], ag.depart, 2] == 0)
        


    # capacity (4)  ## SKIPPING FOR ONE AGENT 
    m.capacity = ConstraintList()
    # movement constraints - for a hex, go through all times - at each time go through all agents, figure out who maters, and put them into constraint
    for cur, data in travel.items():
        for t in range(Tf+1):
            sum = None
            for agid, next, time, layer in data:
                if t < time: continue       # the variable doesn't exist yet and isn't defined for current t

                sum += m.w[agid, cur, t, 1] - m.w[agid, next, t, layer]

            if sum is not None:     # if something has been added to the sum
                m.capacity.add(sum <= 1)

    # takeoff constraints - a form of capacity essentially

    # continuity (5) - for every step in agent, step after must be taken after step is taken and time has passed
    m.cont = ConstraintList()
    for ag in agents:
        for i in range(len(ag.steps) - 1):
            cur_step = ag.steps[i]
            next_step = ag.steps[i+1]

            # iterate through all times except time 0 - set time 0 to == 0 b/c physically impossible but can't tie to a previous step
            for t in range(ag.depart, Tf):     # not Tf+1 b/c of t+1 below
                m.cont.add(m.w[ag.id, next_step, t+1, 1] - m.w[ag.id, cur_step, t, 1] <= 0)

        # do the 0 and 2 layers, for all times
        for t in range(ag.depart, Tf):
            # can't be in 1 until you're in origin 0
            m.cont.add(m.w[ag.id, ag.steps[0], t+1, 1] - m.w[ag.id, ag.steps[0], t, 0] <= 0)

            # can't be in 2 until you're in destination 1
            m.cont.add(m.w[ag.id, ag.steps[-1], t+1, 2] - m.w[ag.id, ag.steps[-1], t, 1] <= 0)


    # time constraints (7) - for every time of every sector of every flight
    # if t is 1, then t' >= t must be 1
    m.time = ConstraintList()
    for ag in agents:
        for s in ag.steps:
            for t in range(ag.depart, Tf):  # not Tf+1 b/c of t+1 below
                    m.time.add(m.w[ag.id, s, t+1, 1] - m.w[ag.id, s, t, 1] >= 0)

        # handle 0 and 2 layer
        for t in range(ag.depart, Tf):
            m.time.add(m.w[ag.id, ag.steps[0], t+1, 0] - m.w[ag.id, ag.steps[0], t, 0] >= 0)
            m.time.add(m.w[ag.id, ag.steps[-1], t+1, 2] - m.w[ag.id, ag.steps[-1], t, 2] >= 0)

    # objective
    def objective(m):
        sum = 0
        for ag in agents:
            end = ag._steps[-1]

            for t in range(ag.depart, Tf):
                if weights:
                    sum += t * ag.cost * (m.w[ag.id, end, t+1, 2] - m.w[ag.id,end, t, 2])
                else:
                    sum += t * (m.w[ag.id, end, t+1, 2] - m.w[ag.id,end, t, 2])
            
            if weights:
                sum -= ag.scheduled * ag.cost
            else:
                sum -= ag.scheduled
        
        return sum

    m.obj = Objective(rule=objective, sense=minimize)

    #solve
    SolverFactory('gurobi').solve(m)
    # m.w.pprint()


    return value(m.obj), m.w




In [14]:
def plot_optimal(layout, coords, agents, vars, Tf = 20, output = True, vis = True):

    if vis: 
        for t in range(Tf):

            locations = {}
            for ag in agents:
                if ag.depart > t: continue

                # find which sector it's in - [id, hex, t, layer]
                cont = True
                index = 0
                while index < len(ag.steps) - 1 and cont:
                    if value(vars[ag.id, ag.steps[index], t, 1]) - value(vars[ag.id, ag.steps[index + 1], t, 1]) == 1:
                        locations[ag.steps[index]] = (ag.id, False, ag.steps[index+1])
                        cont = False
                    index += 1
                
                if cont and value(vars[ag.id, ag.steps[-1], t, 1]) - value(vars[ag.id, ag.steps[-1], t, 2]) == 1:
                    locations[ag.steps[index]] = (ag.id, True, None)
                

                        # locations[ag.loc] = (ag._id, ag.finished, ag.bid[0])


            # for ag in active:
            #     locations[ag.loc] = (ag._id, ag.finished, ag.bid[0])
            radius = 2
            fig, ax = plt.subplots(1, dpi=radius * 25)

            # ax.set(xlim=(-7, 7), ylim=(-7,7))

            for h in coords:

                color = None
                if h in locations.keys():
                    if locations[h][1]: color = "Green"
                    else: color = "Red"
                
                # print(hex_to_pixel(layout, h))
                x, y = hex_to_pixel(layout, h)
                hex = RegularPolygon((x,y), numVertices=6, radius= 1, 
                                        orientation=np.radians(0), 
                                        facecolor= color, alpha=0.2, edgecolor='k')
                ax.add_patch(hex)
                
                if h in locations.keys() and locations[h][2] is not None:
                    x_n, y_n = hex_to_pixel(layout, locations[h][2])
                    dx = x_n - x
                    dy = y_n - y
                    arrow = Arrow(x + dx/4, y + dy/4, dx/2, dy/2, width=0.25)
                    ax.add_patch(arrow)
                    
                # # Also add a text label
                ax.text(x, y, locations[h][0] if h in locations.keys() else '', ha='center', va='center', fontsize="medium")

            ax.set_aspect('equal')
            ax.autoscale()
            plt.show()


    if output: 
        for ag in agents:
            end = ag._steps[-1]

            sum = 0
            for t in range(ag.depart, Tf-1):
                sum += t * (value(vars[ag.id, end, t+1, 2]) - value(vars[ag.id, end, t, 2]))
            
            sum -= ag.scheduled

            print("Agent ", ag.id, " costed ", sum, "departure, scheduled, arrived", ag._depart_t, ag._schedule_t, ag._arrival_t, "price", ag._var_cost)   # think of this as extra/delayed costs



In [8]:
grid, agents, schedule = create_random(num_agents = 10, radius = 2, iters = 10, seed = 6)

for i, ag in enumerate(agents):
    ag._id = i

opt, vars = optimal_solver(grid, agents, schedule, Tf=30)
# print(value(vars[0, 1, -1, 0, 7 ,1]))

In [16]:
# grid, agents, schedule = create_connected(radius = 3)
# grid, agents, schedule = create_single()
grid, agents, schedule = create_random(num_agents = 10, radius = 2, iters = 10, seed = 7)
# grid, agents, schedule = create_random(num_agents = 20, radius = 2, iters=10, seed = 2)
# grid, agents, schedule = create_hubspoke(num_agents = [25, 25, 25, 25, 25, 25], radius = 7, seed = 10)
# grid, agents, schedule = create_random(num_agents=124, radius=7, iters=50, operator_flag=True, seed = 100)

for i, ag in enumerate(agents):
    ag._id = i

weights = True
opt, vars = optimal_solver(grid, agents, schedule, Tf=20, weights=weights)
rev, delay, std_delay, _, _, waits, _, _, _, _, _, _ = simulate(grid, agents, schedule, vis= False, prior="backpressure", output=True, debug=False)

print("Raw Delay", delay)
print("Weighted Delay", waits)
print("Optimal, weights ", weights, opt)


## Try to get plotting
# layout = Layout(layout_pointy, Point(1, 1), Point(0, 0))
# plot_optimal(layout, grid.coords_l, agents, vars, Tf = 20)

# for t in range(30):

#     locations = {}
#     for ag in agents:
#         if ag.depart > t: continue

#         # find which sector it's in - [id, hex, t, layer]
#         cont = True
#         index = 0
#         while index < len(ag.steps) - 1 and cont:
#             if value(vars[ag.id, ag.steps[index], t, 1]) - value(vars[ag.id, ag.steps[index + 1], t, 1]) == 1:
#                 locations[ag.steps[index]] = (ag.id, False, ag.steps[index+1])
#                 cont = False
#             index += 1
        
#         if cont and value(vars[ag.id, ag.steps[-1], t, 1]) - value(vars[ag.id, ag.steps[-1], t, 2]) == 1:
#             locations[ag.steps[index]] = (ag.id, True, None)
        

#                 # locations[ag.loc] = (ag._id, ag.finished, ag.bid[0])


#     # for ag in active:
#     #     locations[ag.loc] = (ag._id, ag.finished, ag.bid[0])
#     radius = 2
#     fig, ax = plt.subplots(1, dpi=radius * 25)

#     # ax.set(xlim=(-7, 7), ylim=(-7,7))

#     for h in grid.coords_l:

#         color = None
#         if h in locations.keys():
#             if locations[h][1]: color = "Green"
#             else: color = "Red"
        
#         # print(hex_to_pixel(layout, h))
#         x, y = hex_to_pixel(layout, h)
#         hex = RegularPolygon((x,y), numVertices=6, radius= 1, 
#                                 orientation=np.radians(0), 
#                                 facecolor= color, alpha=0.2, edgecolor='k')
#         ax.add_patch(hex)
        
#         if h in locations.keys() and locations[h][2] is not None:
#             x_n, y_n = hex_to_pixel(layout, locations[h][2])
#             dx = x_n - x
#             dy = y_n - y
#             arrow = Arrow(x + dx/4, y + dy/4, dx/2, dy/2, width=0.25)
#             ax.add_patch(arrow)
            
#         # # Also add a text label
#         ax.text(x, y, locations[h][0] if h in locations.keys() else '', ha='center', va='center', fontsize="medium")

#     ax.set_aspect('equal')
#     ax.autoscale()
#     plt.show()



# for ag in agents:
#     end = ag._steps[-1]

#     sum = 0
#     for t in range(ag.depart, 30):
#         sum += t * (value(vars[ag.id, end, t+1, 2]) - value(vars[ag.id,end, t, 2]))
    
#     sum -= ag.scheduled

#     print("Agent ", ag.id, " costed ", sum, "departure, scheduled, arrived", ag._depart_t, ag._schedule_t, ag._arrival_t, "price", ag._var_cost)   # think of this as extra/delayed costs



# print(delay)

Time:  11
Priority:  backpressure
Agent  0 operator  None  costed  (27, 9) finished True departure, scheduled, arrived 5 10 11 price 9
Agent  1 operator  None  costed  (3, 6) finished True departure, scheduled, arrived 6 8 10 price 3
Agent  2 operator  None  costed  (8, 0) finished True departure, scheduled, arrived 2 7 7 price 8
Agent  3 operator  None  costed  (8, 8) finished True departure, scheduled, arrived 3 7 8 price 8
Agent  4 operator  None  costed  (5, 0) finished True departure, scheduled, arrived 0 3 3 price 5
Agent  5 operator  None  costed  (3, 3) finished True departure, scheduled, arrived 2 4 5 price 3
Agent  6 operator  None  costed  (0, 0) finished True departure, scheduled, arrived 7 11 11 price 5
Agent  7 operator  None  costed  (5, 10) finished True departure, scheduled, arrived 4 5 7 price 5
Agent  8 operator  None  costed  (0, 16) finished True departure, scheduled, arrived 6 9 11 price 8
Agent  9 operator  None  costed  (9, 9) finished True departure, scheduled,

In [43]:
for k in range(100):

    grid, agents, schedule = create_random(num_agents = 20, radius = 2, iters=10)

    for i, ag in enumerate(agents):
        ag._id = i
        ag._var_cost = 1
        # print(ag._steps[0], ag.loc)
    #     ag.move((ag._steps[0], 0))

    rev, delay, std_delay, _, _, _, _, _, _, _, _, _ = simulate(grid, agents, schedule, vis= False, prior="backpressure", output=False, debug=False)
    print(delay)
    opt = optimal_solver(grid, agents, schedule)

    if delay < opt:
        print("Error")
        break

print("finished")

5
23
22
16
5
10
27
26
18
24
47
12
15
16
32
33
10
13
19
16
15
31
14
10
22
13
15
20
15
19
44
15
13
14
18
2
21
7
44
29
18
15
26
10
22
17
30
3
20
17
33
19
23
12
24
24
28
21
17
19
19
33
15
18
18
23
56
37
11
45
22
39
18
31
11
10
23
10
15
14
22
24
23
9
16
10
13
14
12
53
25
12
12
16
15
12
45
13
15
36
finished


In [29]:
# grid, agents, schedule = create_hubspoke(num_agents = [25, 25, 25, 25, 25, 25], radius = 7, seed = 10)
# grid, agents, schedule = create_random(num_agents = 30, radius = 5, iters=10, seed = 2)
grid, agents, schedule = create_random(num_agents=124, radius=7, iters=50, operator_flag=True, seed = 100)
operator_flag=True
delays = []
optimals = []
for _ in range(30):
    # grid, agents, schedule = create_hubspoke(num_agents = [25, 25, 25, 25, 25, 25], radius = 7, seed = i+100)
    grid, agents, schedule = create_random(num_agents=124, radius=7, iters=50, operator_flag=True, seed = i+100)
    # grid, agents, schedule = create_bimodal(num_agents=126, radius=7, time=50, operator_flag = operator_flag, seed = i + 1000)

    for i, ag in enumerate(agents):
        ag._id = i

    rev, delay, std_delay, _, _, _, _, _, _, _, _, _ = simulate(grid, agents, schedule, vis= False, prior="backpressure", output=False, debug=False)
    delays.append(delay)
    opt = optimal_solver(grid, agents, schedule, Tf = 70)
    optimals.append(opt)

In [30]:
diff = np.array(delays) - np.array(optimals)
print(np.average(diff))
print(np.average(delays))
print(np.average(optimals))

28.933333333333334
79.56666666666666
50.63333333333333


In [96]:
# bimodal_del = delays
# bimodal_opt = optimals

# np.savetxt("bimodal_del", bimodal_del, delimiter=",")
# np.savetxt("bimodal_opt", bimodal_opt, delimiter=",")

In [6]:
Tf = 5
# grid, agents, schedule = create_connected(radius = 3)
grid, agents, schedule = create_single()
# grid, agents, schedule = create_random(num_agents = 3, radius = 3, iters = 10, seed = 6)
for i, ag in enumerate(agents):
    ag._id = i


# Create variables w for all flights, times, sectors (-2 = not taken off, -1 = done, 0->Nf = in flight )
# variables have Agent, Sector (only on path), Time (only after takeoff), Layer (0, 1, 2)
# for each flight iterate
vars = []
departures = {}     # hex : id  where hex is the takeoff and id is agent
travel = {}         # hex: (id, index, layer) where index is where in ag._steps the hex is at


for ag in agents:
    indices = []

    for i, h in enumerate(ag._steps):
        # add to traveled sectors
        if h not in travel: travel[h] = []
        if i + 1 >= len(ag._steps):
            travel[h].append((ag._id, ag._steps[-1], 2))
        else:
            travel[h].append((ag._id, ag._steps[i+1], 1))
        # if i == 0:
        #     travel[h].append((ag._id, ag._steps[0], ))

        # for every time we make variables for the agent, sector, time
        for t in range(ag._depart_t + i + 1, Tf+1):
            indices.append((ag._id, h, t, 1))

    # add depart/arrival variables
    for t in range(ag._depart_t, Tf+1):
        indices.append((ag._id, ag._steps[0], t, 0))
    for t in range(ag._depart_t + len(ag._steps) - 1, Tf+1):
        indices.append((ag._id, ag._steps[-1], t, 2))
    
    vars += indices

    # if ag._id == 2: print(indices)

    # add to departure sectors
    if ag._steps[0] not in departures: departures[ag._steps[0]] = []
    departures[ag._steps[0]].append(ag._id)

# print(vars)

# Create model
m = ConcreteModel()
m.vars = Set(initialize = vars)
m.w = Var(m.vars, domain=Binary)    # indexing is (id, q, r, s, time, layer)

# m.pprint()
# print(departures)
# print(schedule)


# takeoff constraint (2)
m.takeoff = ConstraintList()
def takeoff(m, dep, ids, t):
    sum = None
    for id in ids:
        try: m.w[id, dep, t-1, 1]
        except: continue
        sum += m.w[id, dep, t, 1] - m.w[id, dep, t-1, 1]

    if sum is None: return Constraint.Skip
    return sum <= 1

# for ag in agents:
for dep, ids in departures.items():
    for t in range(1, Tf+1):
        m.takeoff.add(takeoff(m, dep, ids, t))

# m.takeoff.pprint() # verified should be correct


# capacity constraint (4)
# sum over all flights stuck in sec1 but not going to sec2 must be in capacity
m.capacity = ConstraintList()
def capacity(m, h, data, t):
    
    sum = None
    for (id, sec, lay) in data:
        try: m.w[id, h, t, 1]
        except: continue

        try:  m.w[id, sec, t, lay]
        except: continue

        sum += m.w[id, h, t, 1] - m.w[id, sec, t, lay]

    if sum is None: return Constraint.Skip
    return sum <= 1

# for every flight that is at a sector s, check f-s and f-next sector
# but next sector could be dif for everyone, so need to spec carefully - but next sector should be only thing existing
print(travel)
for h, data in travel.items():
    for t in range(0, Tf+1):
        m.capacity.add(capacity(m, h, data, t))

m.capacity.pprint() # verified correct

# connectivity constraint (5)
m.connect = ConstraintList()
for ag in agents:
    for i, h in enumerate(ag._steps):
        if i == len(ag._steps)-1: continue     # skip dealing with i+1, which is end
        h_n = ag._steps[i+1]
        if ag._steps[i+1] == ag._steps[-1]: print('ok')

        for t in range(ag._depart_t+i+ 1, Tf):
            m.connect.add(m.w[ag._id, h_n, t+1, 1] - m.w[ag._id, h, t, 1] <= 0)
    
    # dealing with beginning and end
    init = ag._steps[0]
    end = ag._steps[-1]
    for t in range(ag._depart_t, Tf):
        m.connect.add(m.w[ag._id, init, t+1, 1] - m.w[ag._id, init, t, 0] <= 0)
    for t in range(ag._depart_t + len(ag._steps), Tf):
        m.connect.add(m.w[ag._id, end, t+1, 2] - m.w[ag._id, end, t, 1] <= 0)

# m.connect.pprint()



m.carry = ConstraintList()
for ag in agents:
    for i, h in enumerate(ag._steps):
        for t in range(ag._depart_t+i+ 1, Tf):
            for j in range(t+1, Tf+1):
                m.carry.add(m.w[ag._id, h, j, 1] - m.w[ag._id, h, t, 1] >= 0)
    
    # dealing with beginning and end
    init = ag._steps[0]
    end = ag._steps[-1]
    for t in range(ag._depart_t, Tf):
        m.carry.add(m.w[ag._id, init, t+1, 0] - m.w[ag._id, init, t, 0] >= 0)
    for t in range(ag._depart_t + len(ag._steps), Tf):
        m.carry.add(m.w[ag._id, end, t+1, 2] - m.w[ag._id, end, t, 2] >= 0)

# m.carry.pprint()

# initial and ending states
m.boundary = ConstraintList()
for ag in agents:
    # for t in range(ag._depart_t, Tf+1):
    #     m.boundary.add(m.w[ag._id, ag._steps[0], t, 0] == 1)

    # for t in range(ag._depart_t + len(ag._steps), Tf+1):
    #     print(m.w[ag._id, ag._steps[-1], t, 2])
    m.boundary.add(m.w[ag._id, ag._steps[-1], Tf, 2] == 1)


# Objective
def objective(m):
    sum = 0
    for ag in agents:
        end = ag._steps[-1]
        for t in range(ag._depart_t + len(ag._steps) - 1, Tf):
            # try: m.w[ag._id, end, t+1, 2]
            # except: 'broken'

            # try: m.w[ag._id, end, t, 2]
            # except: 'broken'

            sum += t * (m.w[ag._id, end, t+1, 2] - m.w[ag._id, end, t, 2])
        
        sum -= ag._schedule_t

    
    return sum

m.obj = Objective(rule=objective, sense=minimize)

#solve
SolverFactory('gurobi').solve(m).write()

m.w.pprint()
# capacity(m, h, data, t)
#     sum = None
#     for id, sec, lay in data:
#         sum += m.w[id, h.q, h.r, h.s, t, 1] - m.w[id, sec.q, sec.r, sec.s, t, lay]

# for h, ids in travel.items():
#     for t in range(0, Tf+1):
#         m.capacity.add(m, h, travel, ids, t)

# print(len(grid.coords_l))
# print(travel)


        # neighbors = [(s.q+1, s.r-1, s.s), (s.q-1, s.r+1, s.s),
        #         (s.q, s.r-1, s.s+1), (s.q, s.r-1, s.s-1),
        #         (s.q-1, s.r, s.s+1), (s.q+1, s.r, s.s-1)]


# m.T_set = RangeSet(0, Tf+1)
# m.S_set = RangeSet(0, len(grid.coords_l))   # this is all coord sectors
# m.L_set = RangeSet(0, 3)        # this is the layers of takeoff flight landing
# m.A_set = RangeSet(0, len(agents))

# # m.w = Var(m.A_set, m.S_set, m.L_set, m.T_set, domain=Binary)

# # add constraints

# # takeoff constraint (2)
# # m.T_set_trim = RangeSet(0, Tf)
# # m.takeoff = Constraint(m.S_set, m.T_set_trim, rule= lambda m, sec, t: m.w[:, sec, 0])

# # capacity constraint (4)
# def connections(w, ag):
    

# m.capacity = Constraint(m.S_set, )

# connectivity constraint (5)

# time constraint (7)

# slide to finish constraint

# initial starting points constraints

# goal constraints

{h(3,-3,0): [(0, h(2,-2,0), 1)], h(2,-2,0): [(0, h(1,-1,0), 1)], h(1,-1,0): [(0, h(0,0,0), 1)], h(0,0,0): [(0, h(0,0,0), 2)]}
capacity : Size=11, Index=capacity_index, Active=True
    Key : Lower : Body                              : Upper : Active
      3 :  -Inf : w[0,3,-3,0,2,1] - w[0,2,-2,0,2,1] :   1.0 :   True
      4 :  -Inf : w[0,3,-3,0,3,1] - w[0,2,-2,0,3,1] :   1.0 :   True
      5 :  -Inf : w[0,3,-3,0,4,1] - w[0,2,-2,0,4,1] :   1.0 :   True
      6 :  -Inf : w[0,3,-3,0,5,1] - w[0,2,-2,0,5,1] :   1.0 :   True
     10 :  -Inf : w[0,2,-2,0,3,1] - w[0,1,-1,0,3,1] :   1.0 :   True
     11 :  -Inf : w[0,2,-2,0,4,1] - w[0,1,-1,0,4,1] :   1.0 :   True
     12 :  -Inf : w[0,2,-2,0,5,1] - w[0,1,-1,0,5,1] :   1.0 :   True
     17 :  -Inf :  w[0,1,-1,0,4,1] - w[0,0,0,0,4,1] :   1.0 :   True
     18 :  -Inf :  w[0,1,-1,0,5,1] - w[0,0,0,0,5,1] :   1.0 :   True
     23 :  -Inf :   w[0,0,0,0,4,1] - w[0,0,0,0,4,2] :   1.0 :   True
     24 :  -Inf :   w[0,0,0,0,5,1] - w[0,0,0,0,5,2] :   1.0 :

In [25]:
agents[1]._depart_t
len(agents[1]._steps)

for ag in agents:
    print(ag._schedule_t)

2
3
2
